# Mawqif Arabic Stance Detection 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import string 
import re
import nltk

<H4>Let's take a look at our dataset</H4>

In [2]:
df_train = pd.read_csv('Mawqif_AllTargets_Train.csv')
df_train.head()

,ID,text,target,stance,stance:confidence,against_reason,favor_reason,none_reason,sarcasm,sarcasm:confidence,sentiment,sentiment:confidence,datetime,Date
0,1,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,Women empowerment,Against,0.5116,A_Explicit,NaN,NaN,No,1.0000,Negative,1.0000,2021-01-16 03:19:19+00:00,16/01/2021
1,3,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,Covid Vaccine,NaN,0.4003,NaN,NaN,Not clear,Yes,0.5990,Neutral,0.6180,2022-04-28 11:12:56+00:00,28/04/2022
2,4,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...,Women empowerment,Favor,0.8171,NaN,F_Explicit,NaN,Yes,0.8145,Negative,0.8251,2022-04-02 07:45:42+00:00,02/04/2022
3,6,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في الم...,Digital Transformation,Favor,1.0000,NaN,F_Explicit,NaN,No,1.0000,Positive,0.7531,2022-02-02 18:24:09+00:00,02/02/2022
4,7,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,Digital Transformation,Favor,0.7559,NaN,F_Explicit,NaN,No,1.0000,Neutral,0.8116,2022-03-27 10:36:04+00:00,27/03/2022


In [3]:
df = df_train[['target','text','stance','sentiment','sarcasm']].copy()
df.head()

,target,text,stance,sentiment,sarcasm
0,Women empowerment,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,Against,Negative,No
1,Covid Vaccine,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,NaN,Neutral,Yes
2,Women empowerment,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...,Favor,Negative,Yes
3,Digital Transformation,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في الم...,Favor,Positive,No
4,Digital Transformation,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,Favor,Neutral,No


In [4]:
df.shape

(3502, 5)

In [5]:
df.stance.value_counts()

stance
Favor      2148
Against    1021
Name: count, dtype: int64

In [6]:
df['stance'].fillna('None', inplace=True)
df.head()

,target,text,stance,sentiment,sarcasm
0,Women empowerment,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,Against,Negative,No
1,Covid Vaccine,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,None,Neutral,Yes
2,Women empowerment,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...,Favor,Negative,Yes
3,Digital Transformation,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في الم...,Favor,Positive,No
4,Digital Transformation,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,Favor,Neutral,No


In [7]:
df.stance.value_counts()

stance
Favor      2148
Against    1021
None        333
Name: count, dtype: int64

In [8]:
def stance_classes(x):
    if x == 'Favor':
        return 1
    elif x == 'Against':
        return 0
    return 2

df['stance_classes'] = df['stance'].apply(stance_classes)
df.head()

,target,text,stance,sentiment,sarcasm,stance_classes
0,Women empowerment,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,Against,Negative,No,0
1,Covid Vaccine,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,None,Neutral,Yes,2
2,Women empowerment,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...,Favor,Negative,Yes,1
3,Digital Transformation,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في الم...,Favor,Positive,No,1
4,Digital Transformation,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,Favor,Neutral,No,1


In [9]:
df.target.value_counts()

target
Women empowerment         1190
Covid Vaccine             1167
Digital Transformation    1145
Name: count, dtype: int64

let's remove unneccesary columns

## let's look at some visualization to make everything clear
- This dataset consists of 4,121 tweets in multi-dialectal Arabic. Each tweet is annotated with a stance toward one of three targets: “COVID-19 vaccine,” “digital transformation,” and “women empowerment.” In addition, it is annotated with sentiment and sarcasm polarities.

- The following figure illustrates the labels’ distribution across all targets, and the distribution per target.
  
<img src = stats.png/>

#### All good! now we can handle punctuation in the text column

In [10]:
import unicodedata as ud

for i in df['text'][2]:
     print((i, ud.category(i)))

('ه', 'Lo')
('ذ', 'Lo')
('ا', 'Lo')
(' ', 'Zs')
('م', 'Lo')
('ا', 'Lo')
(' ', 'Zs')
('ي', 'Lo')
('ُ', 'Mn')
('ع', 'Lo')
('ر', 'Lo')
('ّ', 'Mn')
('ف', 'Lo')
(' ', 'Zs')
('ب', 'Lo')
('ـ', 'Lm')
("'", 'Po')
('ف', 'Lo')
('و', 'Lo')
('ب', 'Lo')
('ي', 'Lo')
('ا', 'Lo')
(' ', 'Zs')
('ا', 'Lo')
('ل', 'Lo')
('م', 'Lo')
('ر', 'Lo')
('أ', 'Lo')
('ة', 'Lo')
(' ', 'Zs')
('ا', 'Lo')
('ل', 'Lo')
('م', 'Lo')
('ُ', 'Mn')
('ت', 'Lo')
('م', 'Lo')
('ك', 'Lo')
('ن', 'Lo')
('ة', 'Lo')
("'", 'Po')
(' ', 'Zs')
('آ', 'Lo')
('ف', 'Lo')
('ة', 'Lo')
(' ', 'Zs')
('ف', 'Lo')
('ك', 'Lo')
('ر', 'Lo')
('ي', 'Lo')
('ة', 'Lo')
(' ', 'Zs')
('ت', 'Lo')
('غ', 'Lo')
('ز', 'Lo')
('و', 'Lo')
(' ', 'Zs')
('ع', 'Lo')
('ق', 'Lo')
('و', 'Lo')
('ل', 'Lo')
(' ', 'Zs')
('ب', 'Lo')
('ع', 'Lo')
('ض', 'Lo')
(' ', 'Zs')
('ا', 'Lo')
('ل', 'Lo')
('ر', 'Lo')
('ج', 'Lo')
('ا', 'Lo')
('ل', 'Lo')
(' ', 'Zs')
('ح', 'Lo')
('ي', 'Lo')
('ث', 'Lo')
(' ', 'Zs')
('إ', 'Lo')
('ن', 'Lo')
('ه', 'Lo')
(' ', 'Zs')
('إ', 'Lo')
('ذ', 'Lo')
(' ', 'Zs')
('ر'

In [11]:
def remove_punct(text):
    return text.apply(lambda x: ''.join(t for t in x if not ud.category(t).startswith('P')))

In [12]:
df['text']

0        عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...
1        روح حلل محد يم تطعيم كورونا شف الحرم البارح م...
2       هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...
3       #LEAP22  مؤتمر يجمع اشهر وابرز المؤثرين في الم...
4       خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...
                              ...                        
3497    يا جماعة الخير انا و اهلي حاشتنا كورونا خلصنا ...
3498     للاسف بعض مدراء الدوائر من العقليات القديمة س...
3499     ليس عصر تمكين المرأة . وإنما عصر تدمير الأسرة...
3500    ياجماعه الخير  هل يعقل اليوم جهة خدمية تذهب لا...
3501     التحول الالكتروني يقضي على هذه الفوضى ويرتقي ...
Name: text, Length: 3502, dtype: object

In [13]:
df['text'] = remove_punct(df['text'])
df['text']

0        عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...
1        روح حلل محد يم تطعيم كورونا شف الحرم البارح م...
2       هذا ما يُعرّف بـفوبيا المرأة المُتمكنة آفة فكر...
3       LEAP22  مؤتمر يجمع اشهر وابرز المؤثرين في المج...
4       خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...
                              ...                        
3497    يا جماعة الخير انا و اهلي حاشتنا كورونا خلصنا ...
3498     للاسف بعض مدراء الدوائر من العقليات القديمة س...
3499     ليس عصر تمكين المرأة  وإنما عصر تدمير الأسرة ...
3500    ياجماعه الخير  هل يعقل اليوم جهة خدمية تذهب لا...
3501     التحول الالكتروني يقضي على هذه الفوضى ويرتقي ...
Name: text, Length: 3502, dtype: object

The following function searches for URLs in the text column. If there are any, further preprocessing is needed to ensure the quality of the training text

In [14]:
def search_for_links(text):

    link_pattern = re.compile(r'<a\s+(?:[^>]*?\s+)?href=(["\'])(.*?)\1')
    matches = link_pattern.findall(text)
    
    return bool(matches)

In [15]:
X = df['text'].apply(search_for_links)
if X.any():
    print("Some entries contain links.")
    print(df[X])
else:
    print("No entries contain links.")

No entries contain links.


Good! not URLs are found.<br>
Now let's remove stopwords and do some final text cleaning 

In [16]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('Arabic'))

def remove_stopwords(text):
    
    words = word_tokenize(text)
    filtered_sentence = [word for word in words if word not in stop_words]
    return " ".join(filtered_sentence)
    
df['cleaned_text'] = df['text'].apply(remove_stopwords)
df[['text','cleaned_text']]

,text,cleaned_text
0,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترند...
1,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,روح حلل محد يم تطعيم كورونا شف الحرم البارح مل...
2,هذا ما يُعرّف بـفوبيا المرأة المُتمكنة آفة فكر...,يُعرّف بـفوبيا المرأة المُتمكنة آفة فكرية تغزو...
3,LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في المج...,LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين المجال ا...
4,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,خصوصية البيانات وحمايتها المنظمة مطلب ينبغي تك...
...,...,...
3497,يا جماعة الخير انا و اهلي حاشتنا كورونا خلصنا ...,جماعة الخير انا اهلي حاشتنا كورونا خلصنا العام...
3498,للاسف بعض مدراء الدوائر من العقليات القديمة س...,للاسف مدراء الدوائر العقليات القديمة سبب تعثر ...
3499,ليس عصر تمكين المرأة وإنما عصر تدمير الأسرة ...,عصر تمكين المرأة وإنما عصر تدمير الأسرة والمجت...
3500,ياجماعه الخير هل يعقل اليوم جهة خدمية تذهب لا...,ياجماعه الخير يعقل اليوم جهة خدمية تذهب لانجاز...


In [17]:
'''
import qalsadi.lemmatizer
lem = qalsadi.lemmatizer.Lemmatizer()

def lemmatize_text(text):
    
    return ' '.join([lem.lemmatize(word) for word in text.split()])

df['cleaned_text'] = df['cleaned_text'].apply(lemmatize_text)
df[['text', 'cleaned_text']]
'''

"\nimport qalsadi.lemmatizer\nlem = qalsadi.lemmatizer.Lemmatizer()\n\ndef lemmatize_text(text):\n    \n    return ' '.join([lem.lemmatize(word) for word in text.split()])\n\ndf['cleaned_text'] = df['cleaned_text'].apply(lemmatize_text)\ndf[['text', 'cleaned_text']]\n"

In [18]:
from nltk.stem.isri import ISRIStemmer

st = ISRIStemmer()
def stem_text(text):
    return ' '.join([st.stem(word) for word in text.split()])
df['cleaned_text'] = df['cleaned_text'].apply(stem_text)
df[['text','cleaned_text']]

,text,cleaned_text
0,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,عشن لمع صور ويع تمك رأة يصر رند حكم كثر قمع رأ...
1,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,روح حلل محد يم طعم كورو شف حرم برح لين بدن شرط...
2,هذا ما يُعرّف بـفوبيا المرأة المُتمكنة آفة فكر...,عرف ـفب رأة تمك افة فكر غزو عقل رجل تمك رأة بس...
3,LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في المج...,LEAP22 ؤمر جمع شهر برز ؤثر جال تقن خلل سعد عرض...
4,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,خصص بين وحم نظم طلب نبغ تكون ذرع لسد طرق امم ت...
...,...,...
3497,يا جماعة الخير انا و اهلي حاشتنا كورونا خلصنا ...,جمع خير انا اهل حشت كورو خلص عام اضي طعم طعم ك...
3498,للاسف بعض مدراء الدوائر من العقليات القديمة س...,اسف درء دئر عقل قدم سبب عثر تحل كترو وسس
3499,ليس عصر تمكين المرأة وإنما عصر تدمير الأسرة ...,عصر تمك رأة وإن عصر دمر اسر جمع غبء نسء شعر بذ...
3500,ياجماعه الخير هل يعقل اليوم جهة خدمية تذهب لا...,جمع خير عقل اليوم جهة خدم ذهب جاز عمل تجد لرد ...


### processing the test dataset

In [19]:
df_t = pd.read_csv('Mawqif_AllTargets_Blind Test.csv')
df_t.head()

,ID,text,target
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment


We need to do the same preproccessing steps on the test dataset

In [20]:
df_t['clean_text'] = remove_punct(df_t['text'])
df_t

,ID,text,target,clean_text
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine,البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد ...
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment,انا مع تمكين المرأة اصلا URL
...,...,...,...,...
614,4097,التحول الإلكتروني دا معاه مخاطر إلكترونية كبير...,Digital Transformation,التحول الإلكتروني دا معاه مخاطر إلكترونية كبير...
615,4098,ريم النجم: تطعيم كورونا راح يحولكم زومبي !,Covid Vaccine,ريم النجم تطعيم كورونا راح يحولكم زومبي
616,4099,البيانات هي العنصر الرئيسي و المهم جداً في رحل...,Digital Transformation,البيانات هي العنصر الرئيسي و المهم جداً في رحل...
617,4102,تطعيم انفلونزا مناعة مؤقته تطعيم كورونا مناعة...,Covid Vaccine,تطعيم انفلونزا مناعة مؤقته تطعيم كورونا مناعة...


In [21]:
X2 = df_t['clean_text'].apply(search_for_links)
if X2.any():
    print("Some entries contain links.")
    print(df_t[X2])
else:
    print("No entries contain links.")

No entries contain links.


In [22]:
df_t['clean_text'] = df_t['clean_text'].apply(remove_stopwords)
df_t.head()

,ID,text,target,clean_text
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment,الوضع بالسعوديه يفشل شركات صاحيه مشاهير صاحين ...
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation,الفساد الإداري انهك البلاد ومازلنا نعتمد السند...
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine,البي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد اطب...
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment,انا تمكين المرأة اصلا URL


In [23]:
df_t['clean_text'] = df_t['clean_text'].apply(stem_text)
df_t

,ID,text,target,clean_text
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment,وضع سعد فشل شرك صحي مشاهير صاح وضع الحالي قبح ...
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation,فسد دري انه بلد ازل عتمد سند يدي حصل يرد لبد ت...
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine,الب سي ١٠ دول ورب وقف طعم اوكسفورد طبء كوت طعم...
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation,نصة درس سرع عمل تحل علم كترو الن قرب طلب عهم ج...
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment,انا تمك رأة اصل URL
...,...,...,...,...
614,4097,التحول الإلكتروني دا معاه مخاطر إلكترونية كبير...,Digital Transformation,تحل كترو دا عاه خطر الكترونية كبر عند عدد كفي ...
615,4098,ريم النجم: تطعيم كورونا راح يحولكم زومبي !,Covid Vaccine,ريم نجم طعم كورو يحل زومب
616,4099,البيانات هي العنصر الرئيسي و المهم جداً في رحل...,Digital Transformation,بين عنصر رئس مهم جدا رحل تحل إقتصاد رقم ربم حظ...
617,4102,تطعيم انفلونزا مناعة مؤقته تطعيم كورونا مناعة...,Covid Vaccine,طعم انفلونزا ناع ؤقت طعم كورو ناع ؤقت تفد رضى ...


## Feature extraction and modeling

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.cleaned_text, df.stance_classes, test_size = 0.2)

In [44]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(2801,) (701,) (2801,) (701,)


In [46]:
# applying grid search in order to find the best parameters for the TF-IDF Vectorizer and for the SVM model
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

param_grid = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__min_df': (1, 2, 3),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'svm__C': np.logspace(-3, 2, 6),
    'svm__kernel': ['linear', 'rbf']
}

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best score: 0.721
Best parameters set:
	svm__C: 1.0
	svm__kernel: 'linear'
	tfidf__max_df: 0.5
	tfidf__min_df: 1
	tfidf__ngram_range: (1, 2)


In [47]:
v = TfidfVectorizer(max_df=0.5, min_df=1, ngram_range=(1, 3))

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<2801x104418 sparse matrix of type '<class 'numpy.float64'>'
	with 168558 stored elements in Compressed Sparse Row format>

In [48]:
X_train_np = X_train_cv.toarray()
X_train_np

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [49]:
X_train_cv.shape

(2801, 104418)

In [50]:
Doc_Term_Matrix = pd.DataFrame(X_train_np,columns= v.get_feature_names_out())
Doc_Term_Matrix.head()

,000,000 امر,000 امر نفذ,01094746990,10,10 دقق,10 دقق تحل,10 سمح,10 سمح فقط,10 صبح,...,ڤيروس وكل يعط,ڨتي,ڨتي يسز,ڨتي يسز سمر,گلم,گلم وصل,گلم وصل عدم,ﻻدر,ﻻدر قضء,ﻻدر قضء مصاريف
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
model = SVC(C = 1.0,kernel = 'linear')
model.fit(X_train_cv, y_train)

SVC(kernel='linear')

In [52]:
X_val_cv = v.transform(X_val)

## Model evaluation

In [53]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_val_cv)

print(classification_report(y_val, y_pred))
print('Accuracy: ', accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.73      0.72       216
           1       0.78      0.88      0.83       413
           2       0.70      0.10      0.17        72

    accuracy                           0.75       701
   macro avg       0.73      0.57      0.57       701
weighted avg       0.75      0.75      0.73       701

Accuracy:  0.7546362339514978


In [54]:
df_t.head()

,ID,text,target,clean_text,stance_class,stance
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment,وضع سعد فشل شرك صحي مشاهير صاح وضع الحالي قبح ...,1,Favor
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation,فسد دري انه بلد ازل عتمد سند يدي حصل يرد لبد ت...,1,Favor
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine,الب سي ١٠ دول ورب وقف طعم اوكسفورد طبء كوت طعم...,0,Against
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation,نصة درس سرع عمل تحل علم كترو الن قرب طلب عهم ج...,1,Favor
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment,انا تمك رأة اصل URL,1,Favor


In [55]:
X_test = df_t.clean_text

In [56]:
X_test_cv = v.transform(X_test)
X_test_cv

<619x104418 sparse matrix of type '<class 'numpy.float64'>'
	with 16944 stored elements in Compressed Sparse Row format>

In [57]:
predictions = model.predict(X_test_cv)

In [58]:
df_t['stance_class'] = pd.DataFrame(predictions)

In [59]:
def stances(x):

    if x == 0:
        return 'Against'
    elif x == 1:
        return 'Favor'
    else:
        return 'None'

df_t['stance'] = df_t['stance_class'].apply(stances)
df_t.head()

,ID,text,target,clean_text,stance_class,stance
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment,وضع سعد فشل شرك صحي مشاهير صاح وضع الحالي قبح ...,1,Favor
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation,فسد دري انه بلد ازل عتمد سند يدي حصل يرد لبد ت...,1,Favor
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine,الب سي ١٠ دول ورب وقف طعم اوكسفورد طبء كوت طعم...,0,Against
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation,نصة درس سرع عمل تحل علم كترو الن قرب طلب عهم ج...,1,Favor
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment,انا تمك رأة اصل URL,0,Against


In [60]:
df_t['stance'].value_counts()

stance
Favor      436
Against    176
None         7
Name: count, dtype: int64

In [62]:
# saving the results as a tsv file
file_path = 'predictions.tsv'

df_t[['ID', 'stance']].to_csv(file_path, sep='\t', index=False)